# Introducción a Pandas

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />

[Pandas](http://pandas.pydata.org/) es un paquete de Python que proporciona estructuras de datos similares a los dataframes de R. Pandas depende de Numpy, la librería que añade un potente tipo matricial a Python. Los principales tipos de datos que pueden representarse con pandas son:

- Datos tabulares con columnas de tipo heterogéneo con etiquetas en columnas y filas.
- Series temporales.

Pandas proporciona herramientas que permiten:

- leer y escribir datos en diferentes formatos: CSV, Microsoft Excel, bases SQL y formato HDF5
- seleccionar y filtrar de manera sencilla tablas de datos en función de posición, valor o etiquetas
- fusionar y unir datos
- transformar datos aplicando funciones tanto en global como por ventanas
- manipulación de series temporales
- hacer gráficas

En pandas existen tres tipos básicos de objetos todos ellos basados a su vez en Numpy:

- Series (listas, 1D),
- DataFrame (tablas, 2D) y
- Panels (tablas 3D).

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'0.21.1'

## 1.  Series

Una serie (o *Series*) Pandas es un arreglo unidimensional de datos indexados. Puede ser creado desde una lista o arreglo como sigue:

In [2]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64

Como vemos en la salida, la Serie contiene una secuencia de valores y una secuencia de índices, las cuales podemos acceder con los atributos `values` y `index`. Los valores son simplemente un familiar arreglo NumPy:

In [3]:
data.values

array([ 0.25,  0.5 ,  0.75,  1.  ])

El índice es un objeto tipo arreglo, con su nombre de tipo igual a `pd.Index`, el que discutiremos en más detalle.

In [4]:
data.index

RangeIndex(start=0, stop=4, step=1)

Al igual que con un arreglo NumPy, los datos pueden ser accedidos por el índice asociado, a través de la notación de brackets o paréntesis cuadrados:

In [5]:
data[1]

0.5

### Series como un arreglo NumPy generalizado

De lo que hemos visto hasta ahora, puede parecer que el objeto `Series` es básicamente intercambiable con un arreglo unidimensional NumPy. **La diferencia esencial es la presencia del índice**: mientras que el arreglo NumPy tiene un índice entero implícitamente definido, usado para acceder a los valores, la serie Pandas tiene un índice explícitamente definido con los valores.

In [6]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

Y el acceso a los ítemes funciona como es esperado:

In [7]:
data['b']

0.5

### Series como diccionario especializado

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

De esta manera, se puede pensar en una `Series`Pandas un poco como una especialización de un diccionario Python. Un diccionario es una estructira que mapea llaves arbitrarias a un conjunto de valores arbitrarios, y una serie es una estructura que mapea llaves tipadas a un conjunto de valores tipados. Este tipado (la exigencia de un tipo definido de dato) es importante: así como el código específico de tipos compilado detrás de un arreglo Numpy lo hace más eficiente que una lista Python para ciertas operaciones, la información de tipo de datos de una serie Pandas la hace mucho más eficiente que los diccionarios Python para ciertas operaciones.

In [8]:
population_dict = {'Arica y Parinacota': 243149,
                   'Antofagasta': 631875,
                   'Metropolitana de Santiago': 7399042,
                   'Valparaiso': 1842880,
                   'Bíobío': 2127902,
                   'Magallanes y Antártica Chilena': 165547}
population = pd.Series(population_dict)
population

Antofagasta                        631875
Arica y Parinacota                 243149
Bíobío                            2127902
Magallanes y Antártica Chilena     165547
Metropolitana de Santiago         7399042
Valparaiso                        1842880
dtype: int64

You can notice the indexes were sorted lexicographically. That's the default behaviour in Pandas

In [9]:
population['Arica y Parinacota']

243149

Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [10]:
population['Metropolitana':'Valparaíso']

Metropolitana de Santiago    7399042
Valparaiso                   1842880
dtype: int64

## 2. DataFrame

La siguiente estructura fundamental en Pandas es el **`DataFrame`**. Como el objeto `Series` discutido en la sección anterior, el `DataFrame` puede pensarse ya sea como una generalización del arreglo NumPy, o como una especialización de un diccionario Python. Lo miraremos desde ambas perspectivas.

### DataFrame como un arreglo NumPy generalizado

Si una `Series` es el análogo de un arreglo unidimensional con índices flexibles, un `DataFrame` es el análogo de un arreglo bidimensional con índices de fila flexible y nombres de columna flexibles.

In [11]:
# Area in km^2
area_dict = {'Arica y Parinacota': 16873.3,
             'Antofagasta': 126049.1,
             'Metropolitana de Santiago': 15403.2,
             'Valparaiso': 16396.1,
             'Bíobío': 37068.7,
             'Magallanes y Antártica Chilena': 1382291.1}
area = pd.Series(area_dict)
area

Antofagasta                        126049.1
Arica y Parinacota                  16873.3
Bíobío                              37068.7
Magallanes y Antártica Chilena    1382291.1
Metropolitana de Santiago           15403.2
Valparaiso                          16396.1
dtype: float64

Ahora que tenemos esto junto con la serie de población de antes, podemos utilizar un diccionario para construir un único objeto bidimensional que contenga esta información:

In [12]:
regions = pd.DataFrame({'population': population,
                       'area': area})
regions

,area,population
Antofagasta,126049.1,631875
Arica y Parinacota,16873.3,243149
Bíobío,37068.7,2127902
Magallanes y Antártica Chilena,1382291.1,165547
Metropolitana de Santiago,15403.2,7399042
Valparaiso,16396.1,1842880


In [13]:
regions.index

Index(['Antofagasta', 'Arica y Parinacota', 'Bíobío',
       'Magallanes y Antártica Chilena', 'Metropolitana de Santiago',
       'Valparaiso'],
      dtype='object')

In [14]:
regions.columns

Index(['area', 'population'], dtype='object')

### DataFrame como diccionario especializado

Similarmente, podemos pensar el `DataFrame` como la especialización de un diccionario. Donde un diccionario mapea una llave a un valor, un `DataFrame` mapea un nombre de columna a una serie de datos de columna. Por ejemplo, preguntar por el atributo 'área' retorna el objeto Serie conteniendo las áreas que vimos antes:

In [15]:
regions['area']

Antofagasta                        126049.1
Arica y Parinacota                  16873.3
Bíobío                              37068.7
Magallanes y Antártica Chilena    1382291.1
Metropolitana de Santiago           15403.2
Valparaiso                          16396.1
Name: area, dtype: float64

### Construyendo objetos DataFrame
Un `DataFrame` Pandas puede ser construido de una variedad de formas. Veremos algunos ejemplos.

### Desde un único objeto `Series`
Un `DataFrame` es una colección de objetos `Series`, y un `DataFrame` de una sóla columna puede ser construido de una serie individual:

In [16]:
pd.DataFrame(population, columns=['poblacion'])

,poblacion
Antofagasta,631875
Arica y Parinacota,243149
Bíobío,2127902
Magallanes y Antártica Chilena,165547
Metropolitana de Santiago,7399042
Valparaiso,1842880


### Desde un diccionario de objetos `Series`
Como vimos antes, un `DataFrame` puede ser construido a partir de un diccionario de objetos `Series` también:

In [17]:
pd.DataFrame({'poblacion': population,
              'area': area}, columns=['poblacion', 'area'])

,poblacion,area
Antofagasta,631875,126049.1
Arica y Parinacota,243149,16873.3
Bíobío,2127902,37068.7
Magallanes y Antártica Chilena,165547,1382291.1
Metropolitana de Santiago,7399042,15403.2
Valparaiso,1842880,16396.1


## 3. Leyendo un archivo CSV y haciendo operaciones comunes Pandas

In [ ]:
regiones_file='data/chile_regiones.csv'
provincias_file='data/chile_provincias.csv'
comunas_file='data/chile_comunas.csv'

regiones=pd.read_csv(regiones_file, header=0, sep=',')

provincias=pd.read_csv(provincias_file, header=0, sep=',')
comunas=pd.read_csv(comunas_file, header=0, sep=',')

In [ ]:
print('regiones table: ', regiones.columns.values.tolist())
print('provincias table: ', provincias.columns.values.tolist())
print('comunas table: ', comunas.columns.values.tolist())

In [ ]:
regiones.head()

In [ ]:
provincias.head()

In [ ]:
comunas.describe()

In [ ]:
regiones_provincias=pd.merge(regiones, provincias, how='outer')
regiones_provincias.head()

In [ ]:
provincias_comunas=pd.merge(provincias, comunas, how='outer')
provincias_comunas.head()

In [ ]:
regiones_provincias_comunas=pd.merge(regiones_provincias, comunas, how='outer')
regiones_provincias_comunas.index.name='ID'
regiones_provincias_comunas.head()

In [ ]:
regiones_provincias_comunas.to_csv('data/chile_demographic_merge.csv', index=False)

In [ ]:
regiones_provincias_comunas.loc[3]

## 4. Loading ful dataset

In [ ]:
data_file='data/chile_demographic.csv'
data=pd.read_csv(data_file, header=0, sep=',')
data.head()

In [ ]:
# Podemos ordenar el dataframe usando el campo Poblacion

data_sort=data.sort_values('Poblacion')
data_sort.head()

In [ ]:
# Podemos ordenarlo de mayor a menor

data_sort=data.sort_values('Poblacion', ascending=False)
data_sort.head()

In [ ]:
(data.groupby(data['Region'])['Poblacion','Superficie'].sum())

In [ ]:
x=data.groupby(data['Region'])
x['Poblacion','Superficie'].sum()

In [ ]:
(data.groupby(data['Region'])['Poblacion','Superficie'].sum()) \
    .sort_values(['Poblacion'])